In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import json
from development.datasets.osdg_dataset import load_osdg_data
from development.datasets.uclmodules_dataset import load_uclmodules_data
from development.datasets.videscription_dataset import load_videscription_data
from development.train_model import fine_tune_transformer
from development.models.Bert import Bert
from development.models.RoBERTa import RoBERTa
from development.models.ALBERT import ALBERT
from development.preprocessing import remove_stop_words
from development.utils import parse_label

In [3]:
with open('config.json', 'r') as file:
    CONFIG = json.load(file)
    dev_config = CONFIG['development']


In [4]:
osdg_data = load_osdg_data(dev_config['osdg_data_path'], training=True, filter_agreement=False)

In [11]:
ucl_data = load_uclmodules_data(dev_config['uclmodules_data_path'])

In [5]:
videscription_data = load_videscription_data(
    dev_config['videscription_data_path']
)

In [ ]:
bert = Bert()

In [ ]:
result = fine_tune_transformer(
    bert.model,
    bert.tokenizer,
    bert.tokenizer_args,
    data=osdg_data,
    dev_config=dev_config
)

In [20]:
import torch

In [82]:
bert = Bert('./development/weights/Bert/checkpoint-3000/')

In [29]:
long_text = 'The present document provides an initial assessment of the impacts of the voluntary commitments registered as part of the 2017 United Nations Conference to Support the Implementation of Sustainable Development Goal 14: Conserve and sustainably use the oceans, seas and marine resources for sustainable development (Ocean Conference) and in the years since. There are at present 1,628 commitments in the registry of voluntary commitments. The assessment of the impacts of those commitments was greatly hindered by the lack of reporting by many holders of voluntary commitments. Progress reports have been provided on only 393 of the 1,628 commitments, which translates to a reporting proportion of 24 per cent. That assessment has relied on a number of supplementary materials to increase the available information, but regardless remains limited in its ability to undertake a quantitative analysis of collective impacts and to provide a full picture of the impacts of all of the commitments. From the available information, it can be concluded that the voluntary commitments have collectively furthered the attainment of Sustainable Development Goal 14. That is particularly evident in the extensive new area conserved in marine protected areas by the voluntary commitments collectively (currently calculated to be 3.3 million km2, an area slightly larger than the land area of India), and the large number of grassroots initiatives that have collected marine litter from beaches and under water, resulting in both massive amounts of litter removed and awareness raised about marine environmental issues. Bans on single-use plastics have been successfully implemented by governments, resulting in noticeable differences in the appearance of the local environment. In ocean science, networks of scientific cooperation have had a global reach, and have resulted in innovative efforts in capacity development and technology transfer. The heightened awareness of ocean acidification is evident in the increasing number of acidification action plans developed by both local and national governments. Many previously novel approaches, including blue carbon and habitat restoration, are now becoming mainstream, and new technologies are being pioneered to develop artificial reefs, or to transplant and grow coral. New technologies are also being used to make ocean sectors more sustainable, for example by measurably reducing pollution and emissions in shipping and by reducing the environmental impacts of marine aquaculture. The volume of catch certified as sustainable has increased globally, including for small-scale and artisanal fisheries, and efforts are being made to strengthen regional fishery bodies, and provide for increasing collaboration between them and regional seas programmes. In that regard, regional bodies are developing new approaches to working together and to enabling cross-sectoral ocean governance. Innovative financing mechanisms have been developed, including in the context of blue economies, and many new and traditional funders are supporting marine conservation and sustainable use. All of those advances come in the context of continued decline in ocean biodiversity and a decreasing number of sustainable fisheries. Thus, while progress is being made, it is likely not made at a scale that is large enough to make a real difference for the ocean and its biodiversity. The present assessment provides information about some potential gaps in the voluntary commitments that could be further addressed. In reflecting on lessons learned from the commitments, the importance of partnerships and networking are highlighted as ways of scaling up successful initiatives, and to bring in new ideas and develop capacity where it is needed. The importance of sustainable, shock-proof and long-term funding is also highlighted as essential for achieving Goal 14. Finally, it is recommended that future commitments incorporate simple monitoring, with a baseline and agreed-upon metrics. That will make it easier to assess their impacts on both their beneficiaries and on Goal 14 as a whole'

In [86]:
prediction = bert.predict_long_text(long_text)
parse_label(prediction)

'Life Below Water'

In [ ]:
cls_report = bert.evaluate(osdg_data['train'])
print(cls_report)

In [9]:
cls_report = bert.evaluate(osdg_data['valid'])
print(cls_report)

A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'


              precision    recall  f1-score   support

           0       0.87      0.88      0.87       410
           1       0.86      0.93      0.89       368
           2       0.92      0.97      0.95       403
           3       0.92      0.94      0.93       561
           4       0.95      0.93      0.94       650
           5       0.90      0.93      0.92       422
           6       0.91      0.91      0.91       457
           7       0.75      0.73      0.74       227
           8       0.88      0.83      0.85       316
           9       0.84      0.72      0.78       305
          10       0.91      0.91      0.91       342
          11       0.90      0.89      0.89       166
          12       0.89      0.93      0.91       316
          13       0.93      0.92      0.93       171
          14       0.92      0.88      0.90       322
          15       0.99      0.99      0.99       817

    accuracy                           0.91      6253
   macro avg       0.90   

In [ ]:
cls_report = bert.evaluate(osdg_data['test'])
print(cls_report)